# CI/CD with GitHub Actions

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/batmanvane/molt-shield/blob/main/notebooks/cicd_github_actions.ipynb)

**Automate testing and deployment** with GitHub Actions.

This notebook shows developers how to:
- Set up automated testing
- Run security scans in CI
- Build and push Docker images
- Deploy to container registries

## 1. Overview

In [ ]:
print("""
CI/CD Pipeline Overview
=======================

  Push Code   →   Test Suite   →   Build Docker   →   Deploy
  ─────────       ──────────       ──────────────       ───────
      │                │                  │               │
      ▼                ▼                  ▼               ▼
  Git Push       Unit Tests       Image Build      Docker Hub
                  Security         Multi-arch       GHCR
                  Lint             Scan
""")

## 2. GitHub Actions Workflows

In [ ]:
import os

# Create .github/workflows directory
os.makedirs('.github/workflows', exist_ok=True)

# CI Workflow
CI_WORKFLOW = '''
name: CI

on:
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.12'

      - name: Install dependencies
        run: |
          pip install -r requirements.txt
          pip install pytest pytest-cov

      - name: Run tests
        run: pytest tests/ -v --cov=src

  lint:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.12'

      - name: Run ruff
        run: pip install ruff && ruff check src/
'''

with open('.github/workflows/ci.yml', 'w') as f:
    f.write(CI_WORKFLOW)

print("✓ Created .github/workflows/ci.yml")

## 3. Docker Build and Push

In [ ]:
# Docker Build Workflow
DOCKER_WORKFLOW = '''
name: Docker

on:
  push:
    branches: [main]
    tags: ['v*']

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3

      - name: Login to Docker Hub
        uses: docker/login-action@v3
        with:
          username: ${{ secrets.DOCKER_USERNAME }}
          password: ${{ secrets.DOCKER_PASSWORD }}

      - name: Build and push
        uses: docker/build-push-action@v5
        with:
          context: .
          file: ./Dockerfile.prod
          push: true
          tags: |
            ${{ secrets.DOCKER_USERNAME }}/molt-shield:latest
            ${{ secrets.DOCKER_USERNAME }}/molt-shield:${{ github.sha }}
'''

with open('.github/workflows/docker.yml', 'w') as f:
    f.write(DOCKER_WORKFLOW)

print("✓ Created .github/workflows/docker.yml")

## 4. Create Dockerfile

In [ ]:
# Create Dockerfile
DOCKERFILE = '''
# Molt-Shield Production Dockerfile
FROM python:3.12-slim

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    libxml2-dev \
    libxslt1-dev \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy source
COPY src/ ./src/
COPY config/ ./config/

# Create directories
RUN mkdir -p /app/data/input /app/data/output /app/vault

# Expose MCP server port
EXPOSE 3000

# Run MCP server
CMD ["python", "-m", "src.cli", "serve"]
'''

with open('Dockerfile.prod', 'w') as f:
    f.write(DOCKERFILE)

print("✓ Created Dockerfile.prod")

## 5. GitHub Secrets

In [ ]:
print("""
GitHub Secrets Setup
====================

To enable Docker push, add these secrets in your GitHub repo:

1. Go to: Settings → Secrets and variables → Actions
2. Add new repository secrets:

   DOCKER_USERNAME    - Your Docker Hub username
   DOCKER_PASSWORD    - Your Docker Hub password or access token

Optional: For GHCR instead of Docker Hub:

   GHCR_TOKEN         - GitHub Container Registry token
""")

## 6. View Created Files

In [ ]:
import os

print("=== Created Files ===\n")
for root, dirs, files in os.walk('.github'):
    for f in files:
        path = os.path.join(root, f)
        print(f"  {path}")

print("\n=== Dockerfile.prod ===")
with open('Dockerfile.prod', 'r') as f:
    print(f.read()[:500] + "...")

## Summary

You now have complete CI/CD automation:

### Created Files
- `.github/workflows/ci.yml` - Unit tests and linting
- `.github/workflows/docker.yml` - Docker build and push
- `Dockerfile.prod` - Production Docker image

### Workflows
- **CI**: Runs on every push/PR to main
- **Docker**: Builds and pushes on version tags

### Next Steps
- Add security scanning (Trivy, Bandit)
- Add container scanning
- Set up automatic version releases
- Add Helm chart for Kubernetes deployment